In [2]:
import sys
import ast
import pandas as pd

In [2]:
listOfEvents = []
input_file = "out2.txt"
with open(input_file,"r") as f:
    for line in f:
        listOfEvents.append(ast.literal_eval(line))
df = pd.DataFrame.from_dict(listOfEvents)
df["ReadAddr"]=""
df["WriteAddr"]=""



In [39]:

df.dtypes

starttime           int64
address            object
size               object
type               object
cudatime           object
cudatimeoverlap    object
eventid             int64
ReadAddr           object
WriteAddr          object
dtype: object

In [40]:
# df.loc[df["kernelid"]==0,["ReadAddr"]]="anything"
# df[df["kernelid"]==0]["ReadAddr"]

In [45]:
nvbit_file = "../mem_trace_multilayer.txt"
with open(nvbit_file,"r") as f:
    for line in f:
        if "Read" in line:
            data = line.split(":")[1]
            data = data.split(",")
            kernelid = int(data[1])
            readAddr = [e.strip() for e in data[2:] if e != "0x0"]
            #print(readAddr)
            df.loc[(df["type"]!="allocate") & (df["eventid"]==kernelid),["ReadAddr"]]=str(readAddr)
        if "Write" in line:
            data = line.split(":")[1]
            data = data.split(",")
            kernelid = int(data[1])
            writeAddr = [e.strip() for e in data[2:] if e != "0x0"]
            #print(writeAddr)
            df.loc[(df["type"]!="allocate") & (df["eventid"]==kernelid),["WriteAddr"]]=str(writeAddr)
            


['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8d82600000']
['0x7f8e13a00000']
['0x7f8d82600000']
['0x7f8d82600000']
['0x7f8d82600000']
['0x7f8d82600000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8d82600000']
['0x7f8d82600000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8d82600000']
['0x7f8e13a00000', '0x7f8d82600000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
[]
['0x7f8e13a00000']
[]
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13c00000', '0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e13a00000']
['0x7f8e

In [3]:
listOfEvents = []
input_file_nvbit = "out2_nvbit.txt"
with open(input_file_nvbit,"r") as f:
    for line in f:
        listOfEvents.append(ast.literal_eval(line))
df_realaddr = pd.DataFrame.from_dict(listOfEvents)

In [17]:
def get_pagesAllocated(addr, size):
    if size < 0:
        return ""
    else:
        pages_to_return = []
        maxAddr = addr + size
        pageAddr = addr // 4096 *4096
        while pageAddr <= maxAddr:
            pages_to_return.append(pageAddr)
            pageAddr += 4096
    return pages_to_return

In [5]:
df_realaddr['address'] = df_realaddr['address'].apply(lambda x:int(x,0))
df_realaddr['size'] = df_realaddr['size'].astype('int')
df_realaddr[['address','size']]


,address,size
0,184576128,8
1,184662848,8
2,1475545408,400000
3,184665792,8
4,184665792,-8
...,...,...
77,140245754445824,2048
78,140245731639296,1644032
79,140245731639296,-1644032
80,140248192397824,512


In [18]:
df_realaddr['col_3'] = df_realaddr.apply(lambda x: str(get_pagesAllocated(x['address'], x['size'])), axis=1)

In [19]:
df_realaddr['col_3']

0                                           [184573952]
1                                           [184659968]
2     [1475543040, 1475547136, 1475551232, 147555532...
3                                           [184664064]
4                                                      
                            ...                        
77                                    [140245754445824]
78    [140245731639296, 140245731643392, 14024573164...
79                                                     
80                                    [140248192397312]
81                                                     
Name: col_3, Length: 82, dtype: object

In [51]:
realaddr = df_realaddr["address"]
for i, ra in enumerate(realaddr):
    df.loc[(df["type"]=="allocate") & (df["eventid"] == i),["address"]]=ra

In [52]:
df.to_csv(input_file.split('.')[0]+".csv")